In [1]:
"""This script compares the overlap in the different holo states of the complexes"""

import logging
import os
from fepa.core.ensemble_handler import EnsembleHandler
from fepa.utils.file_utils import load_config
from fepa.utils.path_utils import load_abfe_paths_for_compound, load_paths_for_compound
from fepa.utils.md_utils import (
    check_bp_residue_consistency,
)
from fepa.flows.torsions_flow import torsions_analysis_workflow
from fepa.core.visualizers import (
    DimRedVisualizer,
    plot_eigenvalues,
    compute_histograms,
    plot_jsd_histograms,
    plot_pca_components,
)
import mdaencore as encore
from fepa.core.dim_reducers import PCADimReducer, UMAPDimReducer
from fepa.utils.dimred_utils import cluster_pca
from fepa.core.analyzers import compute_relative_entropy
from fepa.utils.feature_utils import (
    convert_features_df_w_components_to_angles,
    convert_features_df_w_angles_to_w_components,
)
import seaborn as sns
import re
import matplotlib.pyplot as plt

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)


# Load configuration
config_path = os.path.join("../../config/config.json")
config = load_config(config_path)
parent_output_dir = "../../wdir/data/"
analysis_output_dir = os.path.join(
    parent_output_dir, "analysis_p6_comparing_ligand_binding_modes_w_components"
)
if not os.path.exists(analysis_output_dir):
    os.makedirs(analysis_output_dir)

/biggin/b211/reub0138/miniforge3/envs/fepa/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/biggin/b211/reub0138/miniforge3/envs/fepa/lib/python3.10/site-packages/Bio/Application/__init__.py:39: BiopythonDeprecationWarning: The Bio.Application modules and modules relying on it have been deprecated.

Due to the on going maintenance burden of keeping command line application
wrappers up to date, we have decided to deprecate and eventually remove these
modules.

We instead now recommend building your command line and invoking it directly
with the subprocess module.
  warnings.warn(
/biggin/b211/reub0138/miniforge3/envs/fepa/lib/python3.10/site-packages/MDAnalysis/analysis/hbonds/hbond_autocorrel.py:54: DeprecationWarning: This module was moved to MDAnalysis.analysis.hydrogenbonds.hbond_autocorrel; hbonds.h

In [2]:
# Creating van_list and leg_window_list
# van_list = [i for i in range(1, 4)]
# leg_window_list = (
#     [f"coul.{i:02}" for i in range(0, 4)]
# [f"coul.{i:02}" for i in range(0, 11)]
# + [f"vdw.{i:02}" for i in range(0, 12)]
# + [f"rest.{i:02}" for i in range(0, 11)]
# )
# cmps_of_interest = ["48951", "47594", "49599", "52542", "47821"]
# Prepare paths
# path_dict = load_abfe_paths_for_compound(
#     config,
#     cmp="42922",
#     bp_selection_string="name CA and resid " + config["pocket_residues_string"],
#     van_list=van_list,
#     leg_window_list=leg_window_list,
#     apo=False,
# )
path_dict = load_paths_for_compound(
    config,
    cmp="42922",
    bp_selection_string="name CA and resid " + config["pocket_residues_string"],
    apo=False,
)
logging.info("Path dict: %s", path_dict)
# Load trajectories
ensemble_handler = EnsembleHandler(path_dict)
ensemble_handler.make_universes()
check_bp_residue_consistency(ensemble_handler.get_universe_dict())

2025-04-27 16:48:29,927 - INFO - Path dict: {'42922_van_1': {'pdb': '/biggin/b211/reub0138/Projects/orexin/deflorian_set_1_j13_v1/OX2_42922/vanilla/npt.gro', 'xtc': '/biggin/b211/reub0138/Projects/orexin/deflorian_set_1_j13_v1/OX2_42922/vanilla/prod.xtc', 'tpr': '/biggin/b211/reub0138/Projects/orexin/deflorian_set_1_j13_v1/OX2_42922/vanilla/prod.tpr', 'bp_selection_string': 'name CA and resid 12  54  57  58  59  60  61  62  64  65  66  71  77  78  81  82  83  84  85  86  89  90  135  138  141  142  161  162  163  174  175  178  182  232  235  236  238  239  242  254  261  264  265  266  268  269'}, '42922_nvt': {'pdb': '/biggin/b211/reub0138/Projects/orexin/deflorian_set_1_j13_v1/OX2_42922/vanilla_rep_3/nvt.gro', 'xtc': '/biggin/b211/reub0138/Projects/orexin/deflorian_set_1_j13_v1/OX2_42922/vanilla_rep_3/nvt.xtc', 'tpr': '/biggin/b211/reub0138/Projects/orexin/deflorian_set_1_j13_v1/OX2_42922/vanilla_rep_3/nvt.tpr', 'bp_selection_string': 'name CA and resid 12  54  57  58  59  60  61  6

True

In [3]:
# Using the flow
sel = "resname unk"
universe_dict = ensemble_handler.get_universe_dict()
universe_list = list(universe_dict.values())
names_list = list(universe_dict.keys())
# Cluster the binding poses
cluster_collection = encore.cluster(
    ensembles=universe_list, select=sel, superimposition_subset=sel
)
print(cluster_collection)

2025-04-27 16:48:50,773 - INFO - Gromacs version   : b'VERSION 2021.4'
2025-04-27 16:48:50,773 - INFO - tpx version       : 122
2025-04-27 16:48:50,773 - INFO - tpx generation    : 28
2025-04-27 16:48:50,774 - INFO - tpx precision     : 4
2025-04-27 16:48:50,774 - INFO - tpx file_tag      : b'release'
2025-04-27 16:48:50,774 - INFO - tpx natoms        : 64578
2025-04-27 16:48:50,775 - INFO - tpx ngtc          : 2
2025-04-27 16:48:50,775 - INFO - tpx fep_state     : 0
2025-04-27 16:48:50,775 - INFO - tpx lambda        : 0.0
2025-04-27 16:48:51,173 - INFO - The attribute(s) masses, types have already been read from the topology file. The guesser will only guess empty values for this attribute, if any exists. To overwrite it by completely guessed values, you can pass the attribute to the force_guess parameter instead of the to_guess one
2025-04-27 16:48:51,175 - INFO - There is no empty types values. Guesser did not guess any new values for types attribute
2025-04-27 16:48:51,175 - INFO -

<ClusterCollection with 77 clusters>


In [ ]:
# Print the cluster membership
for i, cluster in enumerate(cluster_collection):
    print(f"Cluster {i}:")
    print(cluster.metadata)
    print(cluster.metadata["ensemble_membership"].shape)


Cluster 0:
{'ensemble_membership': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3])}
Cluster 1:
{'ensemble_membership': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 3, 3])}
Cluster 2:
{'ensemble_membership': array([1, 1, 1, 3, 3, 3, 3, 3])}
Cluster 3:
{'ensemble_membership': array([1, 1, 1, 1, 1, 1, 1, 1, 1])}
Cluster 4:
{'ensemble_membership': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3])}
Cluster 5:
{'ensemble_membership': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 3])}
Cluster 6:
{'ensemble_membership': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3])}
Cluster 7:
{'ensemble_membership': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1])}
Cluster 8:
{'ensemble_membership': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3])}
Cluster 9:
{'ensemble_membership': array([1, 1, 1, 

In [5]:
cluster_collection

<ClusterCollection with 77 clusters>

In [ ]:
cluster_collection.metadata

In [9]:
type(cluster_collection)

mdaencore.clustering.ClusterCollection.ClusterCollection

In [17]:
# Print the cluster membership
for i, cluster in enumerate(cluster_collection):
    if i<5:
        print(f"Cluster {i}:")
        print('Metadata:',cluster.metadata)
        print('elements:',cluster.elements)
        print('ID:',cluster.id)
        print('Centroid:',cluster.centroid)


Cluster 0:
Metadata: {'ensemble_membership': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3])}
elements: [  1   9  22  32  60  65  70  73  78  90 120 124 165 369 912]
ID: 0
Centroid: 73
Cluster 1:
Metadata: {'ensemble_membership': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 3, 3])}
elements: [  2  19  27  39  40  47  59  68  69  72  75  76  84  91  95  97 106 108
 110 178 213 224 239 242 250 270 284 306 334 501 510 858]
ID: 1
Centroid: 110
Cluster 2:
Metadata: {'ensemble_membership': array([1, 1, 1, 3, 3, 3, 3, 3])}
elements: [ 17  37 128 666 697 698 705 772]
ID: 2
Centroid: 128
Cluster 3:
Metadata: {'ensemble_membership': array([1, 1, 1, 1, 1, 1, 1, 1, 1])}
elements: [130 319 415 416 417 471 485 490 492]
ID: 3
Centroid: 130
Cluster 4:
Metadata: {'ensemble_membership': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3])}
elements: [ 99 147 155 181 188 203 206 210 218 248 292 329 330 360 377 389 877]
ID: 4
Centroid: 

In [21]:
cm = encore.DBSCAN(eps=0.5, min_samples=5, algorithm='auto', leaf_size=30)

In [ ]:
# Using the flow
sel = "resname unk"
cluster_collection = encore.cluster(
    ensembles=universe_list, select=sel, superimposition_subset=sel, method=cm
)

print(cluster_collection)

2025-04-27 17:00:46,993 - INFO - Gromacs version   : b'VERSION 2021.4'
2025-04-27 17:00:46,994 - INFO - tpx version       : 122
2025-04-27 17:00:46,994 - INFO - tpx generation    : 28
2025-04-27 17:00:46,994 - INFO - tpx precision     : 4
2025-04-27 17:00:46,995 - INFO - tpx file_tag      : b'release'
2025-04-27 17:00:46,995 - INFO - tpx natoms        : 64578
2025-04-27 17:00:46,995 - INFO - tpx ngtc          : 2
2025-04-27 17:00:46,995 - INFO - tpx fep_state     : 0
2025-04-27 17:00:46,996 - INFO - tpx lambda        : 0.0
2025-04-27 17:00:47,513 - INFO - The attribute(s) masses, types have already been read from the topology file. The guesser will only guess empty values for this attribute, if any exists. To overwrite it by completely guessed values, you can pass the attribute to the force_guess parameter instead of the to_guess one
2025-04-27 17:00:47,515 - INFO - There is no empty types values. Guesser did not guess any new values for types attribute
2025-04-27 17:00:47,515 - INFO -

<ClusterCollection with 2 clusters>


In [25]:
import numpy as np
# Print the cluster membership
for i, cluster in enumerate(cluster_collection):
    print(f"Cluster {i}:")
    print('Metadata:',cluster.metadata)
    print('Unique Memebers:',np.unique(cluster.metadata['ensemble_membership']))
    print('elements:',cluster.elements)
    print('ID:',cluster.id)
    print('Centroid:',cluster.centroid)


Cluster 0:
Metadata: {'ensemble_membership': array([1, 1, 1, ..., 4, 4, 4], shape=(1498,))}
Unique Memebers: [1 2 3 4]
elements: [   0    1    2 ... 1501 1502 1503]
ID: 0
Centroid: 0
Cluster 1:
Metadata: {'ensemble_membership': array([4, 4, 4, 4, 4, 4])}
Unique Memebers: [4]
elements: [1085 1091 1092 1094 1102 1112]
ID: 1
Centroid: 1085


In [26]:
names_list

['42922_van_1', '42922_nvt', '42922_van_2', '42922_van_3']

In [ ]:
# Making a cluster list for every frame in the universe list
for i, cluster in enumerate(cluster_collection):
    print(f"Cluster {i}:")
    print(cluster.metadata)
    print(cluster.metadata["ensemble_membership"].shape)
    print(cluster.metadata["ensemble_membership"])
    print(cluster.metadata["ensemble_membership"].shape[0])
    print(cluster.metadata["ensemble_membership"].shape[1])
    print(cluster.metadata["ensemble_membership"][0])
    print(cluster.metadata["ensemble_membership"][0].shape)